# Wavelet Convolution

- For a signal $X(t)$, the wavelet transform is defined as:

  $X(\psi _{\omega,\sigma}, t) = [\psi _{\omega,\sigma} \ast X](t)$, where the `*` refers to the [convolution](https://mathworld.wolfram.com/Convolution.html) operator.

  A convolution between two vectors $f$ and $g$ is defined as follows:

  $[f \ast g] = \sum_\tau{f(t)h(t-\tau)}$

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from pyeisen import family
from pyeisen.convolve import fconv

### Define a random time-series signal

In [ ]:
fs = 24                           # Sampling frequecy of hours per day
T = np.arange(0, 365*7, 1/fs)     # 7 Years of data
X = np.random.randn(len(T), 1)    # 1 channels of random data

plt.plot(T, X);
plt.xlabel('Time (days)');
plt.ylabel('Signal Amplitude');
plt.show()

### Define a family of wavelets to demonstrate convolution

In [ ]:
n_kernel = 40
c_periods = np.linspace(1/24, 90, n_kernel)     # Define n_kernel center period that are linearly spaced from 1/24 Day (1 hour) to 90 Days.
cycles = np.ones(n_kernel) * 3                  # Assign each kernel a unique number of cycles. Here we define each kernel with 3 cycles.

morlet_fam = family.morlet(
    freqs=1 / c_periods,                        # Specify frequency as the reciprocal of the periods 
    cycles=cycles,
    fs=24.0,                                    # Assume data are sampled hourly (24 samples per day).
    n_win=6.5
)

print('Number of Kernels Defined: {}'.format(morlet_fam_3['kernel'].shape[0]))

### Convolve wavelets with test signal

- Measure complex-valued, time-varying wavelet coefficients of the signal using wavelet convolution.

In [ ]:
print('Convolving X with morlet_fam, using the following parameters...')
display(morlet_fam['params'])
print(' ')
X_wv = fconv(morlet_fam['kernel'].T, X)
print('Done.\n')

print('Kernel shape (time x kernels): {}'.format(morlet_fam['kernel'].T.shape))
print('Signal shape (time x channels): {}'.format(X.shape))
print('Convolved signal shape (time x kernels x channels): {}'.format(X_wv.shape))

- Use Euler's Method to extract the instantaneous amplitude and phase angle of each wavelet component from the convolved signal. 

In [ ]:
X_wv_amplitude = np.abs(X_wv)
X_wv_phase = np.angle(X_wv)

print('Amplitude component shape (time x kernels x channels): {}'.format(X_wv_amplitude.shape))
print('Phase component shape (time x kernels x channels): {}'.format(X_wv_phase.shape))

- Plot the amplitude and phase over time

In [ ]:
wv_periods = (1/morlet_fam['params']['freqs'])

plt.figure(figsize=(6,4), dpi=300)
ax = plt.subplot(211)
mat = ax.matshow(X_wv_amplitude[:, :, 0].T, aspect=500)
ax.xaxis.set_ticks_position('bottom')
ax.set_xticks(np.arange(0, X_wv_amplitude.shape[0], 10000))
ax.set_xticklabels([int(t) for t in T[np.arange(0, X_wv_amplitude.shape[0], 10000).astype(int)]])
ax.set_xlabel('Signal Time (days)')

ax.set_yticks(np.arange(0, X_wv_amplitude.shape[1], 9))
ax.set_yticklabels([int(per) for per in wv_periods[np.arange(0, X_wv_amplitude.shape[1], 9).astype(int)]])
ax.set_ylabel('Wavelet Cycles (days)')
plt.colorbar(mat, ax=ax, label='Wavelet Amplitude')
plt.show()

- Calculate and plot time-averaged wavelet power from time-varying wavelet amplitude

In [ ]:
X_wv_pow = np.mean(X_wv_amplitude**2, axis=0)

wv_periods = (1/morlet_fam['params']['freqs'])

plt.figure()
ax = plt.subplot(111)
mat = ax.plot(wv_periods, X_wv_pow)
ax.set_xlabel('Wavelet Cycles (days)')
ax.set_ylabel('Wavelet Power')
plt.show()